<a href="https://colab.research.google.com/github/ayyanar7/2023/blob/main/FirstGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [25]:
torch.__version__

'2.6.0+cu124'

In [20]:
!pip3 install torchvision

Defaulting to user installation because normal site-packages is not writeable


In [19]:
!pip3 install tqdm

Defaulting to user installation because normal site-packages is not writeable


In [18]:
!pip3 install pytorch_fid

Defaulting to user installation because normal site-packages is not writeable


In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.spectral_norm import spectral_norm
# from __future__ import print_function
import argparse
import os

import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np
# from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")



In [30]:
torch.__version__
# np.__version__
# pytorch_fid.__version__
# pytorch_image_generation_metrics.__version__



'2.6.0+cu124'

In [35]:
!pip3 install pytorch-fid

Defaulting to user installation because normal site-packages is not writeable


In [40]:
from pytorch_fid import fid_score
# from pytorch-gan-metrics import get_inception_score, ImageDataset
# from pytorch_image_generation_metrics import get_inception_score, ImageDataset

ModuleNotFoundError: No module named 'pytorch_fid'

In [38]:
!pip install pytorch-image-generation-metrics

Defaulting to user installation because normal site-packages is not writeable


In [41]:
def DiffAugment(x, policy='', channels_first=True):
    if policy:
        if not channels_first:
            x = x.permute(0, 3, 1, 2)
        for p in policy.split(','):
            for f in AUGMENT_FNS[p]:
                x = f(x)
        if not channels_first:
            x = x.permute(0, 2, 3, 1)
        x = x.contiguous()
    return x


def rand_brightness(x):
    x = x + (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) - 0.5)
    return x


def rand_saturation(x):
    x_mean = x.mean(dim=1, keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) * 2) + x_mean
    return x


def rand_contrast(x):
    x_mean = x.mean(dim=[1, 2, 3], keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) + 0.5) + x_mean
    return x


def rand_translation(x, ratio=0.125):
    shift_x, shift_y = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    translation_x = torch.randint(-shift_x, shift_x + 1, size=[x.size(0), 1, 1], device=x.device)
    translation_y = torch.randint(-shift_y, shift_y + 1, size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(x.size(2), dtype=torch.long, device=x.device),
        torch.arange(x.size(3), dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + translation_x + 1, 0, x.size(2) + 1)
    grid_y = torch.clamp(grid_y + translation_y + 1, 0, x.size(3) + 1)
    x_pad = F.pad(x, [1, 1, 1, 1, 0, 0, 0, 0])
    x = x_pad.permute(0, 2, 3, 1).contiguous()[grid_batch, grid_x, grid_y].permute(0, 3, 1, 2).contiguous()
    return x


def rand_cutout(x, ratio=0.5):
    cutout_size = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    offset_x = torch.randint(0, x.size(2) + (1 - cutout_size[0] % 2), size=[x.size(0), 1, 1], device=x.device)
    offset_y = torch.randint(0, x.size(3) + (1 - cutout_size[1] % 2), size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(cutout_size[0], dtype=torch.long, device=x.device),
        torch.arange(cutout_size[1], dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + offset_x - cutout_size[0] // 2, min=0, max=x.size(2) - 1)
    grid_y = torch.clamp(grid_y + offset_y - cutout_size[1] // 2, min=0, max=x.size(3) - 1)
    mask = torch.ones(x.size(0), x.size(2), x.size(3), dtype=x.dtype, device=x.device)
    mask[grid_batch, grid_x, grid_y] = 0
    x = x * mask.unsqueeze(1)
    return x


AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'translation': [rand_translation],
    'cutout': [rand_cutout],
}

In [42]:
class BinaryStep(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return (input > 0.).float()

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        zero_index = torch.abs(input) > 1
        middle_index = (torch.abs(input) <= 1) * (torch.abs(input) > 0.4)
        additional = 2 - 4 * torch.abs(input)
        additional[zero_index] = 0.
        additional[middle_index] = 0.4
        return grad_input * additional


class MaskedMLP(nn.Module):
    def __init__(self, in_size, out_size, bias=True, sparse_train=False):
        super(MaskedMLP, self).__init__()
        self.in_size = in_size
        self.out_size = out_size
        self.weight = nn.Parameter(torch.Tensor(out_size, in_size))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_size))
        else:
            self.register_parameter('bias', None)

        self.threshold = nn.Parameter(torch.Tensor(out_size))
        self.step = BinaryStep.apply
        self.mask = None
        self.sparse_train = sparse_train
        self.reset_parameters()

    def reset_parameters(self):
        # nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        # if self.bias is not None:
        #     fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        #     bound = 1 / math.sqrt(fan_in)
        #     nn.init.uniform_(self.bias, -bound, bound)
        nn.init.xavier_uniform(self.weight.data, 1.)
        if self.bias is not None:
            nn.init.zeros_(self.bias)
        with torch.no_grad():
            # std = self.weight.std()
            self.threshold.data.fill_(0)

    def forward(self, input):
        if self.sparse_train:
            # print('sparse training')
            abs_weight = torch.abs(self.weight)
            threshold = self.threshold.view(abs_weight.shape[0], -1)
            abs_weight = abs_weight - threshold
            mask = self.step(abs_weight)
            ratio = torch.sum(mask) / mask.numel()
            # print("keep ratio {:.2f}".format(ratio))
            if ratio <= 0.01:
                with torch.no_grad():
                    # std = self.weight.std()
                    self.threshold.data.fill_(0)
                abs_weight = torch.abs(self.weight)
                threshold = self.threshold.view(abs_weight.shape[0], -1)
                abs_weight = abs_weight - threshold
                mask = self.step(abs_weight)
            self.mask = mask.bool()
        else:
            # print('dense training')
            self.mask = torch.ones_like(self.weight).bool()
        # masked_weight = self.weight * mask
        output = torch.nn.functional.linear(input, self.weight * self.mask, self.bias)
        return output


class MaskedConv2d(nn.Module):
    def __init__(self, in_c, out_c, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True, sparse_train=False):
        super(MaskedConv2d, self).__init__()
        self.in_channels = in_c
        self.out_channels = out_c
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups

        ## define weight
        self.weight = nn.Parameter(torch.Tensor(
            out_c, in_c // groups, *kernel_size
        ))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_c))
        else:
            self.register_parameter('bias', None)
        self.threshold = nn.Parameter(torch.Tensor(out_c))
        self.step = BinaryStep.apply
        self.mask = None
        self.sparse_train = sparse_train
        self.reset_parameters()

    def reset_parameters(self):
        # nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        # if self.bias is not None:
        #     fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        #     bound = 1 / math.sqrt(fan_in)
        #     nn.init.uniform_(self.bias, -bound, bound)
        nn.init.xavier_uniform(self.weight.data, 1.)
        if self.bias is not None:
            nn.init.zeros_(self.bias)
        with torch.no_grad():
            self.threshold.data.fill_(0.)

    def forward(self, x):
        # print(self.sparse_train)
        if self.sparse_train:
            # print('sparse training')
            weight_shape = self.weight.shape
            threshold = self.threshold.view(weight_shape[0], -1)
            weight = torch.abs(self.weight)
            weight = weight.view(weight_shape[0], -1)
            weight = weight - threshold
            mask = self.step(weight)
            mask = mask.view(weight_shape)
            ratio = torch.sum(mask) / mask.numel()
            # print("threshold {:3f}".format(self.threshold[0]))
            # print("keep ratio {:.2f}".format(ratio))
            if ratio <= 0.01:
                with torch.no_grad():
                    self.threshold.data.fill_(0.)
                threshold = self.threshold.view(weight_shape[0], -1)
                weight = torch.abs(self.weight)
                weight = weight.view(weight_shape[0], -1)
                weight = weight - threshold
                mask = self.step(weight)
                mask = mask.view(weight_shape)
            self.mask = mask.bool()
        # masked_weight = self.weight * mask
        else:
            # print('dense training')
            self.mask = torch.ones_like(self.weight).bool()
        # self.weight.retain_grad()
        conv_out = torch.nn.functional.conv2d(x, self.weight * self.mask, bias=self.bias, stride=self.stride,
                                              padding=self.padding, dilation=self.dilation, groups=self.groups)
        return conv_out


def print_layer_keep_ratio(model):
    total = 0.
    keep = 0.
    layer_keep = []
    # print(model.modules())
    for layer in model.modules():
        if isinstance(layer, MaskedMLP):
            total, keep, layer_keep = count_mlp(layer, total, keep, layer_keep)
        elif isinstance(layer, MaskedConv2d):
            total, keep, layer_keep = count_conv(layer, total, keep, layer_keep)
    return keep / total, layer_keep


def count_mlp(layer, total, keep, layer_keep):
    abs_weight = torch.abs(layer.weight)
    threshold = layer.threshold.view(abs_weight.shape[0], -1)
    abs_weight = abs_weight - threshold
    mask = layer.step(abs_weight)
    ratio = torch.sum(mask) / mask.numel()
    total += mask.numel()
    keep += torch.sum(mask)
    # logger.info("Layer threshold {:.4f}".format(layer.threshold[0]))
    layer_ratio = "{}, keep ratio {:.4f}".format(layer, ratio)
    print(layer_ratio)
    layer_keep.append(layer_ratio)
    return total, keep, layer_keep


def count_conv(layer, total, keep, layer_keep):
    weight_shape = layer.weight.shape
    threshold = layer.threshold.view(weight_shape[0], -1)
    weight = torch.abs(layer.weight)
    weight = weight.view(weight_shape[0], -1)
    weight = weight - threshold
    mask = layer.step(weight)
    # print(mask)
    ratio = torch.sum(mask) / mask.numel()
    total += mask.numel()
    keep += torch.sum(mask)
    layer_ratio = "{}, keep ratio {:.4f}".format(layer, ratio)
    print(layer_ratio)
    layer_keep.append(layer_ratio)
    return total, keep, layer_keep


def set_training_mode(model, training_mode):
    for layer in model.modules():
        if isinstance(layer, MaskedMLP) or isinstance(layer, MaskedConv2d):
            layer.sparse_train = training_mode


def sparsity_regularizer(model, lambda_):
    sr_loss = 0
    for layer in model.modules():
        if isinstance(layer, MaskedConv2d) or isinstance(layer, MaskedMLP):
            sr_loss += lambda_ * torch.sum(torch.exp(-layer.threshold))

    return sr_loss

def copy_params(model):
    flatten = deepcopy(list(p.data for p in model.parameters()))
    return flatten


def load_params(model, new_param):
    for p, new_p in zip(model.parameters(), new_param):
        p.data.copy_(new_p)


In [43]:

class ResGenBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
            nn.Upsample(scale_factor=2),
            MaskedConv2d(in_channels, out_channels, (3, 3), stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            MaskedConv2d(out_channels, out_channels, (3, 3), stride=1, padding=1),
        )
        self.shortcut = nn.Sequential(
            nn.Upsample(scale_factor=2),
            MaskedConv2d(in_channels, out_channels, (1, 1), stride=1, padding=0)
        )

    def forward(self, x):
        return self.residual(x) + self.shortcut(x)


class RG_ResGenerator32(nn.Module):
    def __init__(self, z_dim, sparse_train_mode=False):
        super().__init__()
        self.z_dim = z_dim
        self.linear = MaskedMLP(z_dim, 4 * 4 * 256)
        self.sparse_train_mode = sparse_train_mode
        self.blocks = nn.Sequential(
            ResGenBlock(256, 256),
            ResGenBlock(256, 256),
            ResGenBlock(256, 256),
        )
        self.output = nn.Sequential(
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            MaskedConv2d(256, 3, (3, 3), stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        # self.set_training_mode()
        z = self.linear(z)
        z = z.view(-1, 256, 4, 4)
        return self.output(self.blocks(z))


class OptimizedResDisblock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.shortcut = nn.Sequential(
            nn.AvgPool2d(2),
            MaskedConv2d(in_channels, out_channels, (1, 1), 1, 0))
        self.residual = nn.Sequential(
            MaskedConv2d(in_channels, out_channels, (3, 3), 1, 1),
            nn.ReLU(),
            MaskedConv2d(out_channels, out_channels, (3, 3), 1, 1),
            nn.AvgPool2d(2))
        self.initialize()

    def initialize(self):
        for m in self.residual.modules():
            if isinstance(m, MaskedConv2d):
                # init.xavier_uniform_(m.weight, math.sqrt(2))
                # init.zeros_(m.bias)
                spectral_norm(m)
        for m in self.shortcut.modules():
            if isinstance(m, MaskedConv2d):
                # init.xavier_uniform_(m.weight)
                # init.zeros_(m.bias)
                spectral_norm(m)

    def forward(self, x):
        # print(self.residual(x))
        return self.residual(x) + self.shortcut(x)



In [44]:
class ResDisBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=False):
        super().__init__()
        shortcut = []
        if in_channels != out_channels or down:
            shortcut.append(
                MaskedConv2d(in_channels, out_channels, (1, 1), 1, 0))
        if down:
            shortcut.append(nn.AvgPool2d(2))
        self.shortcut = nn.Sequential(*shortcut)

        residual = [
            nn.ReLU(),
            MaskedConv2d(in_channels, out_channels, (3, 3), 1, 1),
            nn.ReLU(),
            MaskedConv2d(out_channels, out_channels, (3, 3), 1, 1),
        ]
        if down:
            residual.append(nn.AvgPool2d(2))
        self.residual = nn.Sequential(*residual)
        self.initialize()

    def initialize(self):
        for m in self.residual.modules():
            if isinstance(m, MaskedConv2d):
                spectral_norm(m)
        for m in self.shortcut.modules():
            if isinstance(m, MaskedConv2d):
                spectral_norm(m)

    def forward(self, x):
        return self.residual(x) + self.shortcut(x)


class RG_ResDiscriminator32(nn.Module):
    def __init__(self, sparse_train_mode=False):
        super().__init__()
        self.model = nn.Sequential(
            OptimizedResDisblock(3, 128),
            ResDisBlock(128, 128, down=True),
            ResDisBlock(128, 128),
            ResDisBlock(128, 128),
            nn.ReLU())
        self.linear = MaskedMLP(128, 1, bias=False)
        self.initialize()
        self.sparse_train_mode = sparse_train_mode

    def initialize(self):
        spectral_norm(self.linear)

    def forward(self, x):
        x = self.model(x).sum(dim=[2, 3])
        x = self.linear(x)
        return x



In [45]:
def main():
    # Create the dataset
    dataset = dset.CIFAR10(root=args.dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(args.image_size),
                               transforms.CenterCrop(args.image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]), download=True, train=True)

    # Make sub-training dataset
    subset = torch.utils.data.Subset(dataset, np.arange(int(len(dataset) * args.data_ratio)))
    # Create the dataloader
    dataloader = torch.utils.data.DataLoader(subset, batch_size=args.batch_size,
                                             shuffle=True, num_workers=args.workers)

    netD = RG_ResDiscriminator32().to(device)
    netG = RG_ResGenerator32(args.noise_size).to(device)

    netG_avg_param = copy_params(netG)

    netG.sparse_train_mode = True
    netD.sparse_train_mode = True

    set_training_mode(netG, netG.sparse_train_mode)
    set_training_mode(netD, netD.sparse_train_mode)

    # Setup Adam optimizers for both G and D
    optimizerD = optim.Adam(netD.parameters(), args.lr, (0, 0.9))
    optimizerG = optim.Adam(netG.parameters(), args.lr, (0, 0.9))

    fixed_noise = torch.randn(64, args.noise_size, device=device)

    print("Starting Training Loop...")
    best_fid = 9999
    fid_record = []

    for epoch in range(1, args.epoch + 1):

        for i, data in enumerate(dataloader, 0):

            netD.zero_grad()
            real_cpu = data[0].to(device)
            b_size = real_cpu.size(0)

            if diffaug_flag:
                real_cpu = DiffAugment(real_cpu, policy=policy)

            output = netD(real_cpu).view(-1)
            errD_real = torch.mean(nn.ReLU(inplace=True)(1.0 - output))

            if netD.sparse_train_mode:
                sr_loss = sparsity_regularizer(netD, args.lambda_)
                errD_real = errD_real + sr_loss

            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, args.noise_size, device=device)
            fake = netG(noise)

            if diffaug_flag:
                fake = DiffAugment(fake, policy=policy)

            output = netD(fake.detach()).view(-1)
            errD_fake = torch.mean(nn.ReLU(inplace=True)(1 + output))
            errD_fake.backward()
            D_G_z1 = output.mean().item()
            errD = errD_real + errD_fake

            if netD.sparse_train_mode:
                for layer in netD.modules():
                    if isinstance(layer, MaskedConv2d) or isinstance(layer, MaskedMLP):
                        try:
                            layer.weight_orig.grad.data = layer.weight_orig.grad.data * layer.mask
                        except:
                            layer.weight.grad.data = layer.weight.grad.data * layer.mask

            optimizerD.step()

            if i % args.n_critic == 0:
                netG.zero_grad()
                noise = torch.randn(b_size, args.noise_size, device=device)
                fake = netG(noise)

                if diffaug_flag:
                    fake = DiffAugment(fake, policy=policy)

                output = netD(fake).view(-1)
                errG = -torch.mean(output)

                if netG.sparse_train_mode:
                    sr_loss = sparsity_regularizer(netG, args.lambda_)
                    errG = errG + sr_loss

                errG.backward()
                D_G_z2 = output.mean().item()

                if netG.sparse_train_mode:
                    for layer in netG.modules():

                        if isinstance(layer, MaskedConv2d) or isinstance(layer, MaskedMLP):
                            try:
                                layer.weight_orig.grad.data = layer.weight_orig.grad.data * layer.mask
                            except:
                                layer.weight.grad.data = layer.weight.grad.data * layer.mask

                optimizerG.step()

                # moving average weight
                for p, avg_p in zip(netG.parameters(), netG_avg_param):
                    avg_p.mul_(0.999).add_(0.001, p.data)

            # Output training stats
            if i % 50 == 0:
                print('[%4d/%4d][%3d/%3d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                      % (epoch, args.epoch, i, len(dataloader),
                         errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Evaluation
        if epoch % args.eva_epoch == 0:

            if netD.sparse_train_mode:
                d_current_keep_ratio, d_layer_keep_ratio = print_layer_keep_ratio(netD)
                print('D keep ratio: %.4f' % d_current_keep_ratio)
                with open('%s/D_keep_ratio_lambda_%s.txt' % (current_model_result_dir, str(args.lambda_)), 'a') as f:
                    f.write('epoch: %d' % epoch + '\n')
                    for x in d_layer_keep_ratio:
                        f.write(x + '\n')
                    f.write('Overall:' + str(d_current_keep_ratio.item()) + '\n')
                    f.write('\n')
            if netG.sparse_train_mode:
                print('--------------')
                g_current_keep_ratio, g_layer_keep_ratio = print_layer_keep_ratio(netG)
                print('G keep ratio: %.4f' % g_current_keep_ratio)
                with open('%s/G_keep_ratio_lambda_%s.txt' % (current_model_result_dir, str(args.lambda_)), 'a') as f:
                    f.write('epoch: %d' % epoch + '\n')
                    for x in g_layer_keep_ratio:
                        f.write(x + '\n')
                    f.write('Overall:' + str(g_current_keep_ratio.item()) + '\n')
                    f.write('\n')

            backup_param = copy_params(netG)
            load_params(netG, netG_avg_param)
            netG.eval()

            fake = netG(fixed_noise).detach().cpu()
            torchvision.utils.save_image(torchvision.utils.make_grid(fake, padding=2, normalize=True),
                                './%s/epoch_%d.png' % (current_model_result_dir, epoch))
            print('Epoch %2d viz images had been saved!' % epoch)

            eva_size = 100
            for iii in tqdm(range(args.eva_size // eva_size), desc='Generating images'):
                Noisee = torch.randn(eva_size, args.noise_size, device=device)
                temp_fake = netG(Noisee)
                for iiii in range(eva_size):
                    torchvision.utils.save_image(temp_fake[iiii].detach(),
                                        '%s/f_%s.png' % (current_model_eva_dir, str(iii * eva_size + iiii)),
                                        normalize=True, range=(-1, 1))
            print('-' * 10 + 'Evaluation Begin' + '-' * 10)

            print('----FID----')
            print('-------------Eva FID------------')
            fid = fid_score.calculate_fid_given_paths([current_model_eva_dir, './dataset/cifar10.test.npz'],
                                                        100, device, 2048)

            print('FID : %.4f' % fid)
            if fid < best_fid:
                best_fid = fid
                print('----IS-----')
                dataset = ImageDataset(current_model_eva_dir, exts=['png', 'jpg'])
                loader = torch.utils.data.DataLoader(dataset, batch_size=100, num_workers=4)
                IS, IS_std = get_inception_score(loader)
                print('Inception Score: {:.2f} +/- {:.2f}'.format(IS, IS_std))

            fid_record.append(fid)

            load_params(netG, backup_param)

            # avg_netG = deepcopy(netG)
            # load_params(avg_netG, netG_avg_param)

            if len(fid_record) >= 5:
                print(fid_record[-5], fid_record[-4], fid_record[-3], fid_record[-2], fid_record[-1])
                average_fid = 0.1 * fid_record[-5] + 0.1 * fid_record[-4] + 0.2 * fid_record[-3] + \
                              0.2 * fid_record[-2] + 0.4 * fid_record[-1]
                print(average_fid)

                if average_fid >= fid:
                    netG.sparse_train_mode = True
                    netD.sparse_train_mode = True
                else:
                    netG.sparse_train_mode = False
                    netD.sparse_train_mode = False
                set_training_mode(netG, netG.sparse_train_mode)
                set_training_mode(netD, netD.sparse_train_mode)

            netG.train()


In [49]:
if __name__ == '__main__':
    model_name = 'RG-SNGAN'
    argparser = argparse.ArgumentParser()
    argparser.add_argument('--epoch', type=int, default=800)
    argparser.add_argument('--batch_size', type=int, default=64)
    argparser.add_argument('--lr', type=float, default=2e-4)
    argparser.add_argument('--workers', type=int, default=4)
    argparser.add_argument('--image_size', type=int, default=32)
    argparser.add_argument('--noise_size', type=int, default=128)
    argparser.add_argument('--dataroot', type=str, default='./dataset')
    argparser.add_argument('--n_critic', type=int, default=5)
    argparser.add_argument('--lambda_', type=float, default=1e-12)
    argparser.add_argument('--data_ratio', type=float, default=1.0)
    argparser.add_argument('--eva_size', type=int, default=10000)
    argparser.add_argument('--eva_epoch', type=int, default=5)
    argparser.add_argument('--diffaug', action='store_true', help='apply DiffAug')

    # args = argparser.parse_args()
    args, unknown = argparser.parse_known_args()
    print(unknown)
    print(args)
    if not os.path.exists(args.dataroot):
        os.makedirs(args.dataroot)

    current_model_result_dir = './%s/result' % model_name

    if not os.path.exists(current_model_result_dir):
        os.makedirs(current_model_result_dir)

    current_model_eva_dir = './%s/eva' % model_name

    if not os.path.exists(current_model_eva_dir):
        os.makedirs(current_model_eva_dir)

    device = "cuda"

    pre_calculated_fid_dir = './dataset/cifar10.test.npz'
    assert os.path.exists(pre_calculated_fid_dir), 'Please put pre-calculated cifar10.test.npz file into ./dataset folder, you can download it from https://drive.google.com/drive/folders/1UBdzl6GtNMwNQ5U-4ESlIer43tNjiGJC'
    print(pre_calculated_fid_dir)
    if args.diffaug == 1:
        policy = 'color,translation,cutout'
        diffaug_flag = True
        print('Diffaug is now enabled')
    else:
        diffaug_flag = False
main()

usage: ipykernel_launcher.py [-h] [--epoch EPOCH] [--batch_size BATCH_SIZE] [--lr LR] [--workers WORKERS]
                             [--image_size IMAGE_SIZE] [--noise_size NOISE_SIZE] [--dataroot DATAROOT]
                             [--n_critic N_CRITIC] [--lambda_ LAMBDA_] [--data_ratio DATA_RATIO] [--eva_size EVA_SIZE]
                             [--eva_epoch EVA_EPOCH] [--diffaug]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Admin\AppData\Roaming\jupyter\runtime\kernel-b3c29120-3770-4a73-8b47-8417b414f43c.json


SystemExit: 2

In [47]:
%tb

SystemExit: 2